In [1]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils
from nltk.tokenize import sent_tokenize
import numpy as np
import re
import pandas as pd
import string
from string import digits
from sklearn.utils import shuffle
print( tf.VERSION )


/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/pyth

1.14.0


## 2) Preprocessing the data

In [29]:
from pathlib import Path
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/" + '/assets/learning/stopword_ja.txt')
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_words_ja = fr.read().splitlines()

In [2]:
df = pd.read_csv('/home/ifte/amiebot_project/MyAI/AI-system/retrieval_Model/processed.csv')

In [3]:
result = shuffle(df)
result = result.iloc[:,1:]

tog = result.iloc[:,:-1].apply(lambda x: ' '.join(x), axis=1)
#tog.to_csv("tog.csv")
input_texts = list(tog.values)

target = result.iloc[:,-1].apply(lambda x: str(x))
target_texts = list(target.values)

In [43]:
result.iloc[:,:-1]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
6331,」,「,浮動,小数点,数,」,「,整数,」,の,...,0,0,0,0,0,0,0,0,0,0
541,に,アクセス,中,チェックイン,は,iOS,の,バックグラウンド,処理,で,...,停止,し,て,いる,状態,コンソール,で,「,利用,禁止
1207,ファイル,の,アップ,ロード,」,を,押し,ます,コンソール,PC,...,日,を,設定,する,場合,その,日時,を,入力,し
6366,で,ユーザ,認証,を,し,ない,全て,の,Windows,PC,...,プルダウンメニュー,の,中,の,「,詳細,設定,」,を,選択
4086,ID,を,受け,渡し,ます,図,登録,ID,の,入力,...,と,コンソール,の,端末,一覧,画面,で,の,端末,アイコ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764,キーパッドアプリ,を,インストール,し,て,も,使え,ない,新しい,メール,...,は,「,Android,Plus,端末,の,構成,プロファイル,の,作成
5067,する,に,は,端末,を,選ん,で,図,を,開き,...,い,ない,とき,は,端末,に,バイパス,を,要求,し
2527,場所,の,共有,を,無効,管理,さ,れ,た,プロファイル,...,の,軌跡,パスワード,文字,列,など,を,表示,し,ない
5579,紛失,端末,を,探す,端末,グループ,の,作成,コンソール,と,...,無線,LAN,の,設定,AirPrint,の,設定,Web,クリップ,の


In [45]:
stop_words = ['」']
stopwords.words('english')

NameError: name 'stopwords' is not defined

In [44]:
result.iloc[:,:-1].apply(lambda x: [item for item in tog if item not in stop_words], axis=1)

6331    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
541     [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
1207    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
6366    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
4086    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
                              ...                        
2764    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
5067    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
2527    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
5579    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
4597    [」 「 浮動 小数点 数 」 「 整数 」 の 中 から 選択 し ます 説明 なぜ この...
Length: 6873, dtype: object

In [32]:
dt = zip(input_texts, target_texts)
lines = pd.DataFrame(dt, columns=['questions','answers'])

In [33]:
lines.questions = lines.questions.apply(lambda x: re.sub('」','', x))
lines.questions = lines.questions.apply(lambda x: re.sub('「','', x))
lines.questions = lines.questions.apply(lambda x: re.sub('URL','', x))
lines.questions = lines.questions.apply(lambda x: re.sub('の','', x))
lines.questions = lines.questions.apply(lambda x: re.sub('て','', x))

In [34]:
#lines.questions
lines.questions


0         浮動 小数点 数   整数   中 から 選択 し ます 説明 なぜ こ 監視 データ ...
1       に アクセス 中 チェックイン は iOS  バックグラウンド 処理 で 行わ れ ます 端...
2       ファイル  アップ ロード  を 押し ます コンソール PC  explore が ポップ...
3       で ユーザ 認証 を し ない 全  Windows PC Windows ME を 除き ...
4       ID を 受け 渡し ます 図 登録 ID  入力 画面 MobiControl エージェン...
                              ...                        
6868    キーパッドアプリ を インストール し  も 使え ない 新しい メール アカウント へ  ...
6869    する に は 端末 を 選ん で 図 を 開き チェック を 外し ます アクティベーション...
6870    場所  共有 を 無効 管理 さ れ た プロファイル  中  アプリ アプリカタログ で ...
6871    紛失 端末 を 探す 端末 グループ  作成 コンソール と 端末 アイコン チェックイン ...
6872    し た カレンダー  ユーザ 名 に 対応 する パスワード SSL を 使用 照会 し た...
Name: questions, Length: 6873, dtype: object

In [10]:
words = [word for word in lines.questions if word not in stop_words_ja]

In [14]:
lines.questions = words
lines.questions

0         浮動 小数点 数   整数   中 から 選択 し ます 説明 なぜ こ 監視 データ ...
1       に アクセス 中 チェックイン は iOS  バックグラウンド 処理 で 行わ れ ます 端...
2       ファイル  アップ ロード  を 押し ます コンソール PC  explore が ポップ...
3       で ユーザ 認証 を し ない 全  Windows PC Windows ME を 除き ...
4       ID を 受け 渡し ます 図 登録 ID  入力 画面 MobiControl エージェン...
                              ...                        
6868    キーパッドアプリ を インストール し  も 使え ない 新しい メール アカウント へ  ...
6869    する に は 端末 を 選ん で 図 を 開き チェック を 外し ます アクティベーション...
6870    場所  共有 を 無効 管理 さ れ た プロファイル  中  アプリ アプリカタログ で ...
6871    紛失 端末 を 探す 端末 グループ  作成 コンソール と 端末 アイコン チェックイン ...
6872    し た カレンダー  ユーザ 名 に 対応 する パスワード SSL を 使用 照会 し た...
Name: questions, Length: 6873, dtype: object

In [15]:
# Add start and end tokens to target sequences
lines.answers = lines.answers.apply(lambda x : '<START> ' + x + ' <END>')
lines

,questions,answers
0,浮動 小数点 数 整数 中 から 選択 し ます 説明 なぜ こ 監視 データ ...,<START> 304 <END>
1,に アクセス 中 チェックイン は iOS バックグラウンド 処理 で 行わ れ ます 端...,<START> 22 <END>
2,ファイル アップ ロード を 押し ます コンソール PC explore が ポップ...,<START> 65 <END>
3,で ユーザ 認証 を し ない 全 Windows PC Windows ME を 除き ...,<START> 308 <END>
4,ID を 受け 渡し ます 図 登録 ID 入力 画面 MobiControl エージェン...,<START> 203 <END>
...,...,...
6868,キーパッドアプリ を インストール し も 使え ない 新しい メール アカウント へ ...,<START> 146 <END>
6869,する に は 端末 を 選ん で 図 を 開き チェック を 外し ます アクティベーション...,<START> 252 <END>
6870,場所 共有 を 無効 管理 さ れ た プロファイル 中 アプリ アプリカタログ で ...,<START> 132 <END>
6871,紛失 端末 を 探す 端末 グループ 作成 コンソール と 端末 アイコン チェックイン ...,<START> 266 <END>


In [16]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( lines.questions + lines.answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 5852


### B) Reading the data from the files

We parse each of the `.yaml` files.

*   Concatenate two or more sentences if the answer has two or more of them.
*   Remove unwanted data types which are produced while parsing the data.
*   Append `<START>` and `<END>` to all the `answers`.
*   Create a `Tokenizer` and load the whole vocabulary ( `questions` + `answers` ) into it.






### C) Preparing data for Seq2Seq model

Our model requires three arrays namely `encoder_input_data`, `decoder_input_data` and `decoder_output_data`.

For `encoder_input_data` :
* Tokenize the `questions`. Pad them to their maximum lquestionsth.

For `decoder_input_data` :
* Tokenize the `answers`. Pad them to their maximum lquestionsth.

For `decoder_output_data` :

* Tokenize the `answers`. Remove the first element from all the `tokenized_answers`. This is the `<START>` element which we added earlier.



In [17]:
questions = lines.questions 
answers = lines.answers

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max([ len(x) for x in tokenized_answers ])
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
#np.save( 'Saved Arrays/enc_in_data.npy' , encoder_input_data )
#np.save( 'Saved Arrays/dec_in_data.npy' , decoder_input_data )
#np.save( 'Saved Arrays/dec_tar_data.npy' , decoder_output_data )


(6873, 100) 100
(6873, 3) 3
(6873, 3, 5852)


## 3) Defining the Encoder-Decoder model
The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.


*   2 Input Layers : One for `encoder_input_data` and another for `decoder_input_data`.
*   Embedding layer : For converting token vectors to fix sized dense vectors. **( Note :  Don't forget the `mask_zero=True` argument here )**
*   LSTM layer : Provide access to Long-Short Term cells.

Working : 

1.   The `encoder_input_data` comes in the Embedding layer (  `encoder_embedding` ). 
2.   The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( `h` and `c` which are `encoder_states` )
3.   These states are set in the LSTM cell of the decoder.
4.   The decoder_input_data comes in through the Embedding layer.
5.   The Embeddings goes in LSTM cell ( which had the states ) to produce seqeunces.

**Important points :**


*   `200` is the output of the GloVe embeddings.
*   `embedding_matrix` is the GloVe embedding which we downloaded earlier.


<center><img style="float: center;" src="https://cdn-images-1.medium.com/max/1600/1*bnRvZDDapHF8Gk8soACtCQ.gif"></center>


Image credits to [Hackernoon](https://hackernoon.com/tutorial-3-what-is-seq2seq-for-text-summarization-and-why-68ebaa644db0).










In [19]:
import tensorflow as tf
shape = 100

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, shape , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( shape , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, shape , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( shape , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    585200      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    585200      input_4[0][0]                    
____________________________________________________________________________________________

## 4) Training the model
We train the model for a number of epochs with `RMSprop` optimizer and `categorical_crossentropy` loss function.

In [20]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=10, epochs=5 ) 
model.save( 'model.h5' ) 


Epoch 1/5
6873/6873 [==============================] - 79s 11ms/sample - loss: 2.1984
Epoch 2/5
6873/6873 [==============================] - 79s 11ms/sample - loss: 1.8164
Epoch 3/5
6873/6873 [==============================] - 76s 11ms/sample - loss: 1.7596
Epoch 4/5
6873/6873 [==============================] - 78s 11ms/sample - loss: 1.6941
Epoch 5/5
6873/6873 [==============================] - 94s 14ms/sample - loss: 1.6325


## 5) Defining inference models
We create inference models which help in predicting answers.

**Encoder inference model** : Takes the question as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the `<start>` tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [25]:
shape = 100

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( shape ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( shape ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


## 6) Talking with our Chatbot

First, we define a method `str_to_tokens` which converts `str` questions to Integer tokens with padding.


In [22]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    #return tokens_list
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [23]:
str_to_tokens("database setting")

array([[3507, 2566,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]], dtype=int32)




1.   First, we take a question as input and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum answer lquestionsth.







In [26]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            ing
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question : database setting
 68 end
Enter question : database
 68 end
Enter question : setting
 68 end
Enter question : Hello
 68 end
Enter question : No
 68 end
Enter question : mobicontrol
 179 end
Enter question : Limits
 180 end


KeyboardInterrupt: 